Generating the runs (tasks):

0-75 - Hard to predict
75-100 - Easy to predict (identifiers for example)

A ideia é ter algo assim:

H (hidden-to predict) = Lat, long (easy)
G (given - input) = name (easy)


H (hidden-to predict) = commercial_flights (hard)
G (given - input) = name (easy)

hard - commercial_flight (input) / easy - name (output)

popularity + change identifiers

--- para gerar a popularidade, teriamos 3 categorias:

high centrality (popular), medium centrality, low centrality


Get three random propertiesproperties ra
three properties (mixing)

Very popular
Medium
Low popularity 

r = 5 -- try different r for how many runs

how many nodes - 3 of 3 categories (central/medium/nocentral)

how many combinations of properties - 4 - 1 easy easy, 1 easy hard, 1 hard easy, 1 hard hard - random combination of properties

180  


### Choosing random number of properties

In [5]:
# Choosing random number of properties
import pandas as pd
import random

props = pd.read_csv('../datasets/ranks/rank_properties.csv')

propsCat = {'easy': props[props['value']>=80]['prop'].to_list(),
'hard': props[props['value']<80]['prop'].to_list()
}

def chooseRandomProperties(n,category):
    return random.sample(propsCat[category], n)


print(chooseRandomProperties(3,'easy'))

['location', 'tz_database_timezone', 'source']


### Choosing entities

In [18]:
# Choosing random eentities
import pandas as pd
import random

entities = pd.read_csv('../datasets/ranks/ranks_pagerank.csv')
entities["quartile"] = pd.qcut(entities["score"], q=3, labels=[1, 2, 3])

entitiesCat = {
    'easy': entities[entities["quartile"]==3]['iata'].to_list(),
    'medium': entities[entities["quartile"]==2]['iata'].to_list(),
    'hard': entities[entities["quartile"]==1]['iata'].to_list(),

}

def chooseRandomEntities(n,category):
    return random.sample(entitiesCat[category], n)



### Constructing H x G

In [34]:
def constructSubsets():
    """
     1 easy easy, 1 easy hard, 1 hard easy, 1 hard hard - random combination of properties
    """

    # Função auxiliar para gerar hidden e given sem sobreposição
    def generate_hidden_given(hidden_difficulty, given_difficulty):

        if hidden_difficulty == 'hard':
           randomizer = 3
        else:
           randomizer = 5

        hidden = chooseRandomProperties(random.randint(1,randomizer), hidden_difficulty)

        given_candidates = chooseRandomProperties(random.randint(1,randomizer), given_difficulty)

        # Remove propriedades que já estão no hidden
        given = [prop for prop in given_candidates if prop not in hidden]

          # Agora obrigamos a ter pelo menos min_given_size
        while len(given) < 2:
            new_prop = chooseRandomProperties(1, given_difficulty)[0]
            if new_prop not in hidden and new_prop not in given:
             given.append(new_prop)

        return hidden, given

    # Agora montamos os conjuntos usando a função
    easy_easyHidden, easy_easyGiven = generate_hidden_given("easy", "easy")
    easy_hardHidden, easy_hardGiven = generate_hidden_given("easy", "hard")
    hard_easyHidden, hard_easyGiven = generate_hidden_given("hard", "easy")
    hard_hardHidden, hard_hardGiven = generate_hidden_given("hard", "hard")

    subsetsTest = {
        'easyeasy': [easy_easyHidden, easy_easyGiven],
        'easyhard': [easy_hardHidden, easy_hardGiven],
        'hardeasy': [hard_easyHidden, hard_easyGiven],
        'hardhard': [hard_hardHidden, hard_hardGiven]
    }


    return subsetsTest

constructSubsets()

{'easyeasy': [['type', 'city', 'altitude'],
  ['country', 'latitude', 'timezone', 'icao', 'longitude']],
 'easyhard': [['city', 'distCity', 'type', 'altitude'],
  ['runwaysLength', 'runways', 'commercial_flights', 'runwaysWidth', 'iata']],
 'hardeasy': [['runwaysLength', 'iata', 'runways'], ['dst', 'distCity']],
 'hardhard': [['commercial_flights'], ['iata', 'runwaysLength']]}

In [36]:
def generateNodes(n=3):
    easyNodes = chooseRandomEntities(n,"easy")
    mediumNodes = chooseRandomEntities(n,"medium")
    hardNodes = chooseRandomEntities(n,"hard")

    return {'easy': easyNodes,'medium': mediumNodes, 'hard': hardNodes}



In [ ]:
def calculatePerplexity():
    pass

def calculateDifficultyTask():
    pass



In [39]:
def generateTasks():
    pass

# Duvida, vou poder passar apenas o contexto, sem info sobre a entidade aeroporto em si?

"""
Uma unica task é composta por t = (n,H,G), onde n= 1 entidade, H sao as hidden properties (para serem preditas), G sao as givens properties (input+context)

a H sempre serao propriedades de um N

teremos um prompt p editado para a task t (n,H,G)

cada task vai ter uma dificudlade entao.. 

task1 = (airportEasy, hiddenEasyEasy, givenEasyeasy) para cada um vai ter as 4 variacoes de hidden e given..
task1 = (airportEasy, hiddenEasyEasy, givenEasyeasy)
task1 = (airportEasy, hiddenEasyEasy, givenEasyeasy)

task2 = (airportMedium, hiddenEasyEasy, givenEasyeasy)
task2 = (airportMedium, hiddenEasyEasy, givenEasyeasy)
task2 = (airportMedium, hiddenEasyEasy, givenEasyeasy)

task3 = (airportHard, hiddenEasyEasy, givenEasyeasy)
task3 = (airportHard, hiddenEasyEasy, givenEasyeasy)
task3 = (airportHard, hiddenEasyEasy, givenEasyeasy)

noContext and context

acho que enviar algo como 'metadados' para depois facilitar la o calculo da dificuldade vai ser interessnate

"""

listOfTasks = []

for diff, entityClass in generateNodes().items():
    for entity in entityClass:
        subsets = constructSubsets()
        for subset in subsets.items():
            metadata = {'difficultyProp':subset[0],"entityDiff:":diff}
            task = (entity, subset[1][0], subset[1][1],metadata)
            listOfTasks.append(task)



print(len(listOfTasks))
for x in listOfTasks:
    print(x)
    

36
('BHM', ['timezone', 'tz_database_timezone', 'icao', 'city', 'latitude'], ['altitude', 'dst', 'name', 'location', 'distCity'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy'})
('BHM', ['tz_database_timezone', 'latitude'], ['iata', 'commercial_flights', 'runways', 'runwaysWidth', 'runwaysLength'], {'difficultyProp': 'easyhard', 'entityDiff:': 'easy'})
('BHM', ['runwaysLength', 'runways', 'iata'], ['city', 'distCity'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'easy'})
('BHM', ['runwaysLength'], ['iata', 'commercial_flights', 'runwaysWidth'], {'difficultyProp': 'hardhard', 'entityDiff:': 'easy'})
('NGB', ['longitude', 'name'], ['icao', 'tz_database_timezone'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy'})
('NGB', ['altitude', 'latitude', 'type', 'longitude', 'timezone'], ['iata', 'runways', 'commercial_flights'], {'difficultyProp': 'easyhard', 'entityDiff:': 'easy'})
('NGB', ['commercial_flights', 'runwaysLength'], ['country', 'distCity'], {'difficultyProp': 'hardea

In [ ]:

def getDataFromNeo4j():
    pass 

def constructInput(entity, given, context=False):
    # construindo o input que vai ser os given + context sim ou nao

    baseInput = f"""
    Here is the entity extracted from the database that you should seek information about:

        input: [INPUT]

        context: [NO_CONTEXT_ADDED].
    """
 
    data = getDataFromNeo4j() # change to the existing function 
    inputUser = baseInput.replace('[INPUT]', data)

    if context:
        contextGenerated = ""
        inputUser = inputUser.replace('[NO_CONTEXT_ADDED]', contextGenerated)

    return inputUser




In [ ]:
def constructPrompt(entity, hidden):
    pass 

In [1]:
context = [
    ('node', [], 'Airport', 'tz_database_timezone', 'Europe/Paris'),
    ('node', [], 'Airport', 'commercial_flights', 1467),
    ('node', [], 'Airport', 'location', 'POINT(0.727605998516 47.4322013855)'),
    ('node', [], 'Airport', 'altitude', 357.0),
    ('node', [], 'Airport', 'dst', 'E'),
    ('node', [], 'Airport', 'type', 'airport'),
    ('node', [], 'Airport', 'runwaysLength', 7887.0),
    ('node', [], 'Airport', 'country', 'France'),
    ('node', [], 'Airport', 'city', 'Tours'),
    ('node', [], 'Airport', 'iata', 'TUF'),
    ('node', [], 'Airport', 'timezone', '1'),
    ('node', [], 'Airport', 'source', 'OurAirports'),
    ('node', [], 'Airport', 'runways', 1.0),
    ('node', [], 'Airport', 'icao', 'LFOT'),
    ('node', [], 'Airport', 'name', 'Tours-Val-de-Loire Airport'),
    ('node', [], 'Airport', 'distCity', 4524.34944026358),
    ('node', [], 'Airport', 'longitude', 0.727605998516),
    ('node', [], 'Airport', 'latitude', 47.4322013855),
    ('node', [], 'Airport', 'runwaysWidth', 148.0),
    ('node', [], 'Airport', 'airport_id', 1376),

    ('node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/London'),
    ('node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 194207),
    ('node', ['ROUTE_TO'], 'Airport', 'location', 'POINT(0.234999999404 51.8849983215)'),
    ('node', ['ROUTE_TO'], 'Airport', 'altitude', 348.0),
    ('node', ['ROUTE_TO'], 'Airport', 'dst', 'E'),
    ('node', ['ROUTE_TO'], 'Airport', 'type', 'airport'),
    ('node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10003.0),
    ('node', ['ROUTE_TO'], 'Airport', 'country', 'United Kingdom'),
    ('node', ['ROUTE_TO'], 'Airport', 'city', 'London'),
    ('node', ['ROUTE_TO'], 'Airport', 'iata', 'STN'),
    ('node', ['ROUTE_TO'], 'Airport', 'timezone', '0'),
    ('node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'),
    ('node', ['ROUTE_TO'], 'Airport', 'runways', 1.0),
    ('node', ['ROUTE_TO'], 'Airport', 'icao', 'EGSS'),
    ('node', ['ROUTE_TO'], 'Airport', 'name', 'London Stansted Airport'),
    ('node', ['ROUTE_TO'], 'Airport', 'distCity', 47103.62927380715),
    ('node', ['ROUTE_TO'], 'Airport', 'longitude', 0.234999999404),
    ('node', ['ROUTE_TO'], 'Airport', 'latitude', 51.8849983215),
    ('node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 151.0),
    ('node', ['ROUTE_TO'], 'Airport', 'airport_id', 548),

    ('node', ['OF'], 'City', 'countrycode', 'fr'),
    ('node', ['OF'], 'City', 'accent_name', 'Tours'),
    ('node', ['OF'], 'City', 'location', 'POINT(0.7039770000000001 47.394836)'),
    ('node', ['OF'], 'City', 'name', 'tours'),
    ('node', ['OF'], 'City', 'longitude', 0.7039770000000001),
    ('node', ['OF'], 'City', 'latitude', 47.394836),
    ('node', ['OF'], 'City', 'country', 'France'),
    ('node', ['OF'], 'City', 'population', 135143.0),

    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.8974738492916),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 496949.47956980375),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 563.2547089778877),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4296'),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'),
    ('edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'FR'),

    ('edge', ['PASSENGERS'], 'PASSENGERS', 'nb_passengers', 40054),
    ('edge', ['PASSENGERS'], 'PASSENGERS', 'nb_seats', 50300),
    ('edge', ['PASSENGERS'], 'PASSENGERS', 'year', 2024),

    ('node', ['OF', 'City', 'IN'], 'Country', 'name', 'France'),
    ('node', ['OF', 'City', 'IN'], 'Country', 'region', 'Europe'),
    ('node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Paris'),
    ('node', ['OF', 'City', 'IN'], 'Country', 'population', 67059887.0),
    ('node', ['OF', 'City', 'IN'], 'Country', 'language', 'French'),
    ('node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.5),
    ('node', ['OF', 'City', 'IN'], 'Country', 'latitude', 46.227638),
    ('node', ['OF', 'City', 'IN'], 'Country', 'longitude', 2.213749),
]

context

[('node', [], 'Airport', 'tz_database_timezone', 'Europe/Paris'),
 ('node', [], 'Airport', 'commercial_flights', 1467),
 ('node', [], 'Airport', 'location', 'POINT(0.727605998516 47.4322013855)'),
 ('node', [], 'Airport', 'altitude', 357.0),
 ('node', [], 'Airport', 'dst', 'E'),
 ('node', [], 'Airport', 'type', 'airport'),
 ('node', [], 'Airport', 'runwaysLength', 7887.0),
 ('node', [], 'Airport', 'country', 'France'),
 ('node', [], 'Airport', 'city', 'Tours'),
 ('node', [], 'Airport', 'iata', 'TUF'),
 ('node', [], 'Airport', 'timezone', '1'),
 ('node', [], 'Airport', 'source', 'OurAirports'),
 ('node', [], 'Airport', 'runways', 1.0),
 ('node', [], 'Airport', 'icao', 'LFOT'),
 ('node', [], 'Airport', 'name', 'Tours-Val-de-Loire Airport'),
 ('node', [], 'Airport', 'distCity', 4524.34944026358),
 ('node', [], 'Airport', 'longitude', 0.727605998516),
 ('node', [], 'Airport', 'latitude', 47.4322013855),
 ('node', [], 'Airport', 'runwaysWidth', 148.0),
 ('node', [], 'Airport', 'airport_id',

In [4]:
# Dos que tem o segundo como [] vazio, tenho que pegar:
# 1. as properties pra saber quais ele tem
# 2. os valores das properties para fazer o ground truth
# 3. o contexto para adicionar depois ou nao

def getEntitiyValuesFromContext(context):
    props = {}
    for row in context:
        if row[0] == 'node' and row[1]==[]:
            props[row[3]] = row[4]

    return props

print(getEntitiyValuesFromContext(context))

{'tz_database_timezone': 'Europe/Paris', 'commercial_flights': 1467, 'location': 'POINT(0.727605998516 47.4322013855)', 'altitude': 357.0, 'dst': 'E', 'type': 'airport', 'runwaysLength': 7887.0, 'country': 'France', 'city': 'Tours', 'iata': 'TUF', 'timezone': '1', 'source': 'OurAirports', 'runways': 1.0, 'icao': 'LFOT', 'name': 'Tours-Val-de-Loire Airport', 'distCity': 4524.34944026358, 'longitude': 0.727605998516, 'latitude': 47.4322013855, 'runwaysWidth': 148.0, 'airport_id': 1376}
